<a href="https://colab.research.google.com/github/cdm106/ML_Final_Project/blob/main/Final_Project_CDM_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project

# Installation and setup of Ollama in the Colab environment.

hello dear
